In [1]:
import concurrent.futures
import importlib
import googleapiclient.discovery
import openai
import os
import requests
import bs4
import traceback
from bs4 import BeautifulSoup
from joblib import Memory
from itertools import islice
openai.organization = "org-RfxFQjm7zizJjdQRALbJZaZB"
openai.api_key = os.environ.get("OPENAI_API_KEY")
modelToUse = "gpt-3.5-turbo"
print(modelToUse)
memory = Memory("/app/cachedata", verbose=0)
googleAPIKey = os.environ.get("GOOGLE_API_KEY")
googleCseId = "368102dbb37ea4659"

import scripts.marbot as marbot

gpt-3.5-turbo


In [2]:
importlib.reload(marbot) # little hack since I jump back and forth
allLinks = marbot.findLinksForAllIndustries(googleAPIKey, googleCseId, modelToUse)

print(allLinks)


Got 10 links for Aircraft manufacturing us challenges: ['https://www.whitehouse.gov/briefing-room/statements-releases/2021/09/09/fact-sheet-biden-administration-advances-the-future-of-sustainable-fuels-in-american-aviation/', 'https://www.aviationpros.com/engines-components/article/53060096/magnetic-mro-us-aviation-supply-chain-challenges-parts-shortages-rising-costs-labor-resource-crunches', 'https://blog.ericson.com/blog/4-key-challenges-in-aircraft-manufacturing-and-maintenance', 'https://www.energy.gov/eere/bioenergy/sustainable-aviation-fuel-grand-challenge', 'https://www.js3global.com/blog/8-challenges-facing-the-aerospace-industry/', 'https://www2.deloitte.com/us/en/pages/manufacturing/articles/aerospace-and-defense-industry-outlook.html', 'https://gama.aero/opportunities-in-ga/aviation-challenge/', 'https://icao.usmission.gov/nasa-to-increase-sustainable-aircraft-manufacturing-with-hicam/', 'https://www.transportation.gov/testimony/aviation-fuels-needs-challenges-and-alternativ

In [3]:
importlib.reload(marbot) # little hack since I jump back and forth
marbot.loadUrls(allLinks)

Fetching https://www.polarismarketresearch.com/industry-analysis/aircraft-manufacturing-market
Saved 153630 chars to /app/testdata/Aircraft-1.html
Fetching https://www.energy.gov/eere/bioenergy/sustainable-aviation-fuel-grand-challenge
Saved 89339 chars to /app/testdata/Aircraft-2.html
Fetching https://www.aviationpros.com/engines-components/article/53060096/magnetic-mro-us-aviation-supply-chain-challenges-parts-shortages-rising-costs-labor-resource-crunches
Saved 106157 chars to /app/testdata/Aircraft-3.html
Fetching https://www.statista.com/markets/407/topic/939/aerospace-defense-manufacturing/
Saved 324901 chars to /app/testdata/Aircraft-4.html
Fetching https://www.gordonbrothers.com/insight/commercial-and-industrial-aircraft-engine-and-parts/
Saved 124753 chars to /app/testdata/Aircraft-5.html
Fetching https://blog.ericson.com/blog/4-key-challenges-in-aircraft-manufacturing-and-maintenance
Saved 89788 chars to /app/testdata/Aircraft-6.html
Fetching https://www.ibisworld.com/industr

In [4]:
importlib.reload(marbot) # little hack since I jump back and forth
filePaths = marbot.scanFiles("/app/testdata")
print(f"Found {len(filePaths)} industries with {sum([len(filePaths[filePath]) for filePath in filePaths])} pages")

Found 15 industries with 392 pages


In [5]:
importlib.reload(marbot) # little hack since I jump back and forth
import traceback ## remove in a little while
industryPageSummaries = {}
industrySummaries = {}
failedPages = []
for industry in sorted(filePaths):
    thisIndustryFilePaths = filePaths[industry]
    pageSummaries = []
    for filePath in thisIndustryFilePaths:
        try:
            pageSummary = marbot.loadAndSummarizePage(filePath, modelToUse)
            pageSummaries.append(pageSummary)
        except Exception as e:
            errorMessage = f"Error on {filePath}: {e}\n{traceback.format_exc()}"
            print(errorMessage)
            failedPages.append(errorMessage)
    industryPageSummaries[industry] = pageSummaries

if len(failedPages) > 0:
    print(f"Failed to summarize {len(failedPages)} pages")
    print("\n\n".join(failedPages))

Loading /app/testdata/Aircraft-3.html (v2)
Read 106157 chars from /app/testdata/Aircraft-3.html
Read 13574 chars of human readable text
Split into 4 batches of max 4000 chars
Submitting batch 0
Submitting batch 1
Submitting batch 2
Submitting batch 3
Summary 0: The main text in the input is about discussions between the U.S. Air Force and the industry regarding their sustainment and supply chain needs in a competitive world. It also mentions FPT Software securing a contract with EASA for utilizing blockchain in aviation, the benefits of upgrading to LED lighting, and the FAA posting a directive for fixing Pratt's Airbus A320 engines. Major trends in the industry include the focus on sustainment and supply chain needs, the adoption of blockchain in aviation, the shift towards LED lighting, and the need for engine fixes and improvements.

Summary 1: The text discusses the current state of the aviation industry and the challenges it is facing, particularly in terms of supply chain constra

In [6]:
for industry in sorted(filePaths):
    print(f"Num and length of page summaries for {industry}: {len(industryPageSummaries[industry])}: {list(len(x) for x in industryPageSummaries[industry])}")
    

Num and length of page summaries for Aircraft: 19: [1270, 1675, 1363, 1308, 1148, 1280, 496, 1959, 1358, 908, 1604, 1657, 1933, 3909, 1904, 975, 1165, 904, 2108]
Num and length of page summaries for Appliance: 21: [1696, 1483, 1342, 1352, 1996, 1764, 2549, 2032, 1690, 977, 1243, 1539, 2140, 1679, 977, 1502, 207, 2563, 1490, 1256, 1840]
Num and length of page summaries for Bakery_bread: 27: [690, 1299, 1461, 1279, 1429, 1589, 437, 2358, 1101, 1420, 545, 1819, 1770, 384, 944, 961, 1326, 1032, 1602, 1037, 2407, 2647, 392, 1631, 2382, 1617, 288]
Num and length of page summaries for Chemical: 24: [1431, 2156, 2589, 1434, 1748, 1538, 1148, 1154, 1701, 889, 1251, 1014, 579, 1308, 2374, 2270, 974, 3652, 1735, 2033, 1434, 1139, 1831, 1852]
Num and length of page summaries for Dairy: 16: [1751, 738, 257, 1161, 2021, 1325, 589, 2237, 869, 1540, 1094, 2770, 1839, 1627, 1158, 1792]
Num and length of page summaries for Electrical: 22: [4529, 1683, 2025, 2254, 1406, 1465, 1625, 1440, 1274, 252, 1765,

In [16]:
importlib.reload(marbot) # little hack since I jump back and forth
industrySummaries = {}
for industry in sorted(industryPageSummaries):
    print(f"\n\n=== Combining {industry}")
    pageSummaries = industryPageSummaries[industry]
    combinedSummary = marbot.combinePageSummaries(pageSummaries, "gpt-4")
    industrySummaries[industry] = combinedSummary
    print(f"Length of overall summary: {len(industrySummaries[industry])}")
    print(f"Overall summary:\n{industrySummaries[industry]}")



=== Combining Aircraft
LEWIS DEBUG v3
Length of overall summary: 2766
Overall summary:
"Supply chain disruptions" (9): The aviation industry is experiencing significant supply chain disruptions due to the pandemic, geopolitical issues, and labor shortages. Disruptions include shortages of raw materials and components, backlogs in the Maintenance, Repair, and Overhaul (MRO) sector, and aircraft order backlogs.

"COVID-19 impact" (7): The pandemic has had a significant effect on the aviation industry, leading to shutdowns, disruptions to supply chains, and delayed deliveries. However, despite these challenges, there is optimism for a recovery as lockdown restrictions lift.

"Sustainability focus" (6): There is an increasing focus on sustainable practices, particularly with respect to fuel, through initiatives such as the production of Sustainable Aviation Fuel (SAF) and the introduction of electric and hybrid-electric aircraft. The goal is to reduce greenhouse gas emissions in the avia

In [19]:
importlib.reload(marbot) # little hack since I jump back and forth


print(f"\n\n=== Combining all")
combinedSummary = marbot.combineIndustrySummaries(list(industrySummaries.values()), "gpt-4")
print(f"Length of overall summary: {len(combinedSummary)}")
print(f"Overall summary:\n{combinedSummary}")



=== Combining all
Length of overall summary: 5282
Overall summary:
"Supply chain disruptions" (93): Most industries, including aviation, household appliances, auto manufacturing, and the bakeries, struggle with significant supply chain disruptions due to the pandemic, geopolitical issues, labor shortages, and changes in consumer habits. These disruptions, including shortages of raw materials and components, slow down production and put a strain on the logistics and operations.

"Technological Innovation/Digital Transformation" (51): Across sectors, there is a trend of technological advancements and digitalization of processes. Industries are adopting technologies like IoT, AI, data analytics, 3D printing, and autonomous systems, fueling market growth and leading to improvement in supply chain management, product development, service delivery, and sustainability efforts.

"Sustainability focus" (44): An increasing focus on sustainability is seen across multiple sectors, with efforts a